In [2]:
!pip install selenium
!pip install bs4
# brew install chromedriver via terminal before continue

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.maximize_window()

In [2]:
def login():
    try:
        driver.get('https://www.finnomena.com/')
        driver.find_element_by_link_text('ลงชื่อเข้าใช้').click()
        driver.find_element_by_id('username-login').send_keys('')
        driver.find_element_by_id('username-login').send_keys('###EMAIL###')
        driver.find_element_by_class_name('next-process').click()
        driver.find_element_by_id('password-login').send_keys('###PASSWORD###')
        driver.find_element_by_class_name('submit-btn').click()
    except Exception as e:
        print(e)
login()

In [45]:
def getStockDetails(name,isYearly):
    # stock-growthrate
    sections = ['stock-growthrate','stock-financial-report','stock-financial-ratio','stock-statistics']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    headers = []
    quarters = []
    data = []
    toggled = False
    try:
        for section in sections:
            obj = {}
            keys = []
            names = []
            headerElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            if len(headers) == 0:
                for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
                    headers.append(div.get_text())
            if not toggled and isYearly:
                toggled = True
                driver.find_element_by_class_name('toggle').click()
                
            for topic in headerElements.find_elements_by_class_name('topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                name = BeautifulSoup(topic.get_attribute('innerHTML'), 'html.parser').get_text()
                keys.append(key)
                names.append(name)
                
                
            contentElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]} {names[i]}"
                obj[key] = values
            
            data.append(obj)
        return (headers,quarters,data)
    except Exception as e:
        print(e) 

In [46]:
import pandas as pd
headers,quartersOrYear,responseData = getStockDetails(name='mint',isYearly=True)


In [47]:
data = {}
for d in responseData:
    data = dict(list(data.items()) + list(d.items()))
data['index'] = quartersOrYear
df = pd.DataFrame(data) 
df = df.iloc[:-1]
df.set_index('index',inplace = True)
df.transpose()

index,2553,2554,2555,2556,2557,2558,2559,2560,2561,2562
Revenue รายได้รวม,"19,089.25","28,331.86","32,993.51","36,936.24","39,787.07","48,149.03","56,972.78","58,643.67","79,328.00","129,889.11"
RevenueYoY การเติบโตต่อปี (%),10.40,48.42,16.45,11.95,7.72,21.02,18.33,2.93,35.27,63.74
NetProfit กำไรสุทธิ,"1,241.10","2,880.14","3,408.55","4,101.38","4,401.79","7,040.16","6,590.00","5,415.40","5,444.77","10,697.93"
NetProfitYoY การเติบโตต่อปี (%),-11.37,132.06,18.35,20.33,7.32,59.94,-6.39,-17.82,0.54,96.48
EarningPerShare กำไรต่อหุ้น (EPS),0.38,0.88,0.94,1.04,1.10,1.60,1.50,1.22,1.18,2.04
EarningPerShareYoY การเติบโตต่อปี (%),-11.02,130.98,6.48,11.44,5.27,45.38,-6.48,-18.48,-3.31,72.81
Asset สินทรัพย์รวม,"32,786.17","40,865.20","50,974.98","59,936.27","74,279.15","95,737.45","108,453.15","118,443.55","267,700.31","254,183.77"
TotalDebt หนี้สินรวม,"18,980.27","25,930.70","31,628.45","33,061.60","44,254.68","59,822.05","67,656.32","68,422.63","184,626.19","168,315.92"
Equity ส่วนของผู้ถือหุ้น,"12,924.08","14,367.35","18,133.39","25,090.41","28,056.71","32,802.52","37,401.73","47,328.59","71,923.74","75,461.25"
PaidUpCapital มูลค่าหุ้นที่เรียกชำระแล้ว,"3,262.34","3,275.22","3,686.77","4,001.36","4,001.56","4,402.31","4,410.37","4,618.91","4,618.91","4,619.00"
